# Aggregating and Combining `pandas` DataFrames

## Objectives

- Use GroupBy objects to organize and aggregate data
- Create pivot tables from DataFrames
- Combine DataFrames by merging, joining, and concatinating

## Set Up

Surprise, surprise... we're still working with the Austin Animal Center Data! Let's start with Outcomes

In [21]:
# Imports

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [22]:
outcomes = pd.read_csv('data/Austin_Animal_Center_Outcomes_041122.csv',
                       parse_dates=['DateTime', 'Date of Birth'])

In [23]:
outcomes.head()

Animal ID   Name            DateTime MonthYear Date of Birth Outcome Type  \
0   A794011  Chunk 2019-05-08 18:20:00  May 2019    2017-05-02    Rto-Adopt   
1   A776359  Gizmo 2018-07-18 16:02:00  Jul 2018    2017-07-12     Adoption   
2   A821648    NaN 2020-08-16 11:38:00  Aug 2020    2019-08-16   Euthanasia   
3   A720371  Moose 2016-02-13 17:59:00  Feb 2016    2015-10-08     Adoption   
4   A674754    NaN 2014-03-18 11:47:00  Mar 2014    2014-03-12     Transfer   

  Outcome Subtype Animal Type Sex upon Outcome Age upon Outcome  \
0             NaN         Cat    Neutered Male          2 years   
1             NaN         Dog    Neutered Male           1 year   
2             NaN       Other          Unknown           1 year   
3             NaN         Dog    Neutered Male         4 months   
4         Partner         Cat      Intact Male           6 days   

                                Breed              Color  
0              Domestic Shorthair Mix  Brown Tabby/White  
1             Chihuahua Shorthair Mix        White/Brown  
2                             Raccoon               Gray  
3  Anatol Shepherd/Labrador Retriever               Buff  
4              Domestic Shorthair Mix       Orange Tabby

In [24]:
outcomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138282 entries, 0 to 138281
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Animal ID         138282 non-null  object        
 1   Name              97090 non-null   object        
 2   DateTime          138282 non-null  datetime64[ns]
 3   MonthYear         138282 non-null  object        
 4   Date of Birth     138282 non-null  datetime64[ns]
 5   Outcome Type      138259 non-null  object        
 6   Outcome Subtype   63199 non-null   object        
 7   Animal Type       138282 non-null  object        
 8   Sex upon Outcome  138281 non-null  object        
 9   Age upon Outcome  138281 non-null  object        
 10  Breed             138282 non-null  object        
 11  Color             138282 non-null  object        
dtypes: datetime64[ns](2), object(10)
memory usage: 12.7+ MB


In [25]:
# Let's create our Age in Days column
outcomes['Calculated Age in Days'] = outcomes['DateTime'].dt.normalize() - outcomes['Date of Birth']

In [26]:
# Grab just the integer here...
outcomes['Calculated Age in Days'] = outcomes['Calculated Age in Days'].dt.days

In [27]:
# Sanity check
outcomes.head()

Animal ID   Name            DateTime MonthYear Date of Birth Outcome Type  \
0   A794011  Chunk 2019-05-08 18:20:00  May 2019    2017-05-02    Rto-Adopt   
1   A776359  Gizmo 2018-07-18 16:02:00  Jul 2018    2017-07-12     Adoption   
2   A821648    NaN 2020-08-16 11:38:00  Aug 2020    2019-08-16   Euthanasia   
3   A720371  Moose 2016-02-13 17:59:00  Feb 2016    2015-10-08     Adoption   
4   A674754    NaN 2014-03-18 11:47:00  Mar 2014    2014-03-12     Transfer   

  Outcome Subtype Animal Type Sex upon Outcome Age upon Outcome  \
0             NaN         Cat    Neutered Male          2 years   
1             NaN         Dog    Neutered Male           1 year   
2             NaN       Other          Unknown           1 year   
3             NaN         Dog    Neutered Male         4 months   
4         Partner         Cat      Intact Male           6 days   

                                Breed              Color  \
0              Domestic Shorthair Mix  Brown Tabby/White   
1             Chihuahua Shorthair Mix        White/Brown   
2                             Raccoon               Gray   
3  Anatol Shepherd/Labrador Retriever               Buff   
4              Domestic Shorthair Mix       Orange Tabby   

   Calculated Age in Days  
0                     736  
1                     371  
2                     366  
3                     128  
4                       6

## Aggregating over DataFrames: `.groupby()`

Those of you familiar with SQL have probably used the GROUP BY command. (And if you haven't, you'll see it very soon!) Pandas has this, too.

The `.groupby()` method is especially useful for aggregate functions applied to the data grouped in particular ways.

In [28]:
# Just using groupby outputs some weird GroupBy object... not helpful
outcomes.groupby('Animal Type').count()

#omg! this is just the long form version of value_counts

Animal ID   Name  DateTime  MonthYear  Date of Birth  \
Animal Type                                                         
Bird               658    146       658        658            658   
Cat              52415  30642     52415      52415          52415   
Dog              77841  65222     77841      77841          77841   
Livestock           25      3        25         25             25   
Other             7343   1077      7343       7343           7343   

             Outcome Type  Outcome Subtype  Sex upon Outcome  \
Animal Type                                                    
Bird                  658              381               658   
Cat                 52411            31973             52415   
Dog                 77827            24897             77840   
Livestock              25               19                25   
Other                7338             5929              7343   

             Age upon Outcome  Breed  Color  Calculated Age in Days  
Animal Type                                                          
Bird                      658    658    658                     658  
Cat                     52415  52415  52415                   52415  
Dog                     77841  77841  77841                   77841  
Livestock                  25     25     25                      25  
Other                    7342   7343   7343                    7343

Once we know we are working with a type of object, it opens up a suite of attributes and methods. One attribute we can look at is `groups`.

In [29]:
# This returns each group indexed by the group name, e.g. 'Bird',
# along with the row indices of each value

outcomes.groupby('Animal Type').groups

{'Bird': [206, 533, 983, 1025, 1282, 1308, 2216, 2254, 2270, 2413, 2517, 2594, 2708, 2774, 3173, 3374, 3643, 3738, 3998, 4019, 4283, 4697, 4761, 4993, 5058, 5200, 5431, 5651, 5843, 6082, 6231, 6335, 6587, 6677, 7028, 7347, 7423, 7980, 8043, 8310, 8326, 8409, 8533, 8917, 9197, 9442, 9752, 9819, 10097, 10159, 10401, 10651, 10730, 11380, 11610, 11668, 11726, 11759, 11765, 11831, 12198, 12411, 12416, 12444, 12467, 12705, 12894, 12970, 13049, 13055, 13087, 13264, 13309, 13315, 13427, 13466, 13669, 13926, 13942, 13955, 13973, 14100, 14123, 14138, 14185, 15106, 15185, 15535, 15545, 15805, 16014, 16189, 16491, 16857, 17164, 17329, 17381, 17417, 18310, 18350, ...], 'Cat': [0, 4, 7, 8, 10, 11, 14, 15, 16, 17, 18, 20, 24, 26, 34, 37, 49, 54, 56, 66, 67, 68, 70, 75, 78, 80, 83, 84, 89, 90, 92, 94, 95, 97, 98, 102, 113, 115, 116, 117, 118, 120, 122, 126, 139, 141, 142, 145, 147, 148, 151, 152, 156, 157, 158, 164, 167, 168, 170, 171, 176, 178, 184, 191, 192, 194, 200, 202, 203, 207, 209, 212, 215, 2

In [30]:
# Same goes for multi-index groupbys
animal_outcome = outcomes.groupby(['Animal Type', 'Outcome Type'])

In [31]:
# .groups outputs a dictionary, so we can access the group names using keys()
animal_outcome.groups.keys()

dict_keys([('Cat', 'Rto-Adopt'), ('Dog', 'Adoption'), ('Other', 'Euthanasia'), ('Cat', 'Transfer'), ('Cat', 'Adoption'), ('Cat', 'Return to Owner'), ('Dog', 'Return to Owner'), ('Dog', 'Transfer'), ('Cat', 'Euthanasia'), ('Other', 'Adoption'), ('Dog', 'Rto-Adopt'), ('Cat', 'Died'), ('Dog', 'Euthanasia'), ('Other', 'Transfer'), ('Bird', 'Adoption'), ('Other', 'Disposal'), ('Other', 'Died'), ('Dog', 'Died'), ('Cat', 'Disposal'), ('Other', 'Return to Owner'), ('Bird', 'Euthanasia'), ('Bird', 'Transfer'), ('Livestock', 'Return to Owner'), ('Dog', 'Missing'), ('Other', 'Relocate'), ('Dog', nan), ('Livestock', 'Adoption'), ('Bird', 'Return to Owner'), ('Dog', 'Disposal'), ('Cat', 'Missing'), ('Bird', 'Disposal'), ('Bird', 'Died'), ('Other', 'Missing'), ('Other', 'Rto-Adopt'), ('Bird', 'Relocate'), ('Bird', 'Missing'), ('Other', nan), ('Livestock', 'Transfer'), ('Cat', 'Relocate'), ('Cat', nan), ('Livestock', 'Died'), ('Livestock', 'Euthanasia')])

In [32]:
# We can then get a specific group, such as cats that were adopted
animal_outcome.get_group(('Cat', 'Adoption'))

Animal ID        Name            DateTime MonthYear Date of Birth  \
7        A689724  *Donatello 2014-10-18 18:52:00  Oct 2014    2014-08-01   
8        A680969       *Zeus 2014-08-05 16:59:00  Aug 2014    2014-06-03   
20       A730621       *Liza 2016-09-10 18:59:00  Sep 2016    2016-05-18   
26       A801106         NaN 2019-08-16 14:05:00  Aug 2019    2019-05-06   
54       A792258      Vesper 2019-04-10 20:53:00  Apr 2019    2016-09-08   
...          ...         ...                 ...       ...           ...   
138220   A854491    *Alibaba 2022-04-09 11:53:00  Apr 2022    2022-01-21   
138242   A854453    *Giselle 2022-04-09 16:54:00  Apr 2022    2021-04-04   
138254   A853741        Kiwi 2022-04-09 19:08:00  Apr 2022    2021-03-23   
138255   A853740   Pineapple 2022-04-09 19:08:00  Apr 2022    2021-03-23   
138257   A854004     Coconut 2022-04-09 19:08:00  Apr 2022    2021-10-22   

       Outcome Type Outcome Subtype Animal Type Sex upon Outcome  \
7          Adoption             NaN         Cat    Neutered Male   
8          Adoption             NaN         Cat    Neutered Male   
20         Adoption             NaN         Cat    Spayed Female   
26         Adoption             NaN         Cat    Neutered Male   
54         Adoption             NaN         Cat    Spayed Female   
...             ...             ...         ...              ...   
138220     Adoption             NaN         Cat    Neutered Male   
138242     Adoption             NaN         Cat    Spayed Female   
138254     Adoption             NaN         Cat    Spayed Female   
138255     Adoption             NaN         Cat    Neutered Male   
138257     Adoption             NaN         Cat    Spayed Female   

       Age upon Outcome                   Breed               Color  \
7              2 months  Domestic Shorthair Mix               Black   
8              2 months  Domestic Shorthair Mix  White/Orange Tabby   
20             3 months  Domestic Shorthair Mix              Calico   
26             3 months      Domestic Shorthair        Orange Tabby   
54              2 years  Domestic Shorthair Mix              Tortie   
...                 ...                     ...                 ...   
138220         2 months      Domestic Shorthair   Cream Tabby/White   
138242           1 year      Domestic Shorthair   Brown Tabby/White   
138254           1 year      Domestic Shorthair               Black   
138255           1 year      Domestic Shorthair         Brown Tabby   
138257         5 months  Domestic Shorthair Mix         Brown Tabby   

        Calculated Age in Days  
7                           78  
8                           63  
20                         115  
26                         102  
54                         944  
...                        ...  
138220                      78  
138242                     370  
138254                     382  
138255                     382  
138257                     169  

[24979 rows x 13 columns]

## Aggregating

Once again, as we will see in SQL, groupby objects are intended to be used with aggregation. In SQL, we will see that our queries that include GROUP BY require aggregation performed on columns.

We can use `.sum()`, `.mean()`, `.count()`, `.max()`, `.min()`, etc. Find a list of common aggregations [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html).

In [33]:
# Test it out
outcomes.groupby(by='Animal Type').median()

Calculated Age in Days
Animal Type                        
Bird                            374
Cat                             111
Dog                             730
Livestock                       383
Other                           366

## Exercise

Use `.groupby()` to find the most recent birth date of each (main) animal type.


In [34]:
# Your code here

outcomes.groupby(by='Animal Type')['Date of Birth'].max()

Animal Type
Bird        2022-01-06
Cat         2022-04-06
Dog         2022-04-03
Livestock   2020-05-28
Other       2022-02-11
Name: Date of Birth, dtype: datetime64[ns]

# Pivoting a DataFrame

## `.pivot_table()`

Those of you familiar with Excel have probably used Pivot Tables. Pandas has a similar functionality.

Grouping by two different columns can be very helpful.

In [35]:
outcomes.groupby(by=['Outcome Type', 'Sex upon Outcome']).agg(['mean','median','sum'])

Calculated Age in Days                  
                                                   mean  median       sum
Outcome Type    Sex upon Outcome                                         
Adoption        Intact Female                422.344047   100.0    578189
                Intact Male                  476.037194   128.0    563152
                Neutered Male                651.127082   325.0  19936209
                Spayed Female                647.370062   288.5  19370607
                Unknown                      389.964286   370.0     76433
Died            Intact Female                350.784195    61.0    115408
                Intact Male                  303.058537    50.0    124254
                Neutered Male               1842.490385  1357.5    191619
                Spayed Female               2100.895833  1717.5    201686
                Unknown                      311.353760   181.0    111776
Disposal        Intact Female                407.277778   139.5     14662
                Intact Male                  735.935484   389.0     45628
                Neutered Male               1987.777778  1465.0     17890
                Spayed Female               2470.166667  2476.5     14821
                Unknown                      449.377907   367.0    231879
Euthanasia      Intact Female               1129.880583   375.0   1239479
                Intact Male                  904.908472   603.0   1196289
                Neutered Male               2228.568925  1826.0   1907655
                Spayed Female               2312.266417  1827.0   1232438
                Unknown                      504.882808   366.0   2675374
Missing         Intact Female                266.800000   156.0      5336
                Intact Male                  340.846154   183.0      8862
                Neutered Male               1188.285714   814.0      8318
                Spayed Female               1262.733333   954.0     18941
                Unknown                      169.250000   131.5       677
Relocate        Intact Female                732.000000   732.0       732
                Neutered Male               1105.000000  1105.0      1105
                Spayed Female                495.000000   374.0      1485
                Unknown                      612.200000   210.5     12244
Return to Owner Intact Female               1063.130712   731.0   3001218
                Intact Male                 1106.971545   732.0   4629355
                Neutered Male               1635.703431  1163.5  14875087
                Spayed Female               1751.054955  1293.5  11662026
                Unknown                      769.637838   368.0    142383
Rto-Adopt       Intact Female               1455.724138   751.0     42216
                Intact Male                 1503.175000   886.0     60127
                Neutered Male               1255.863071   795.5    605326
                Spayed Female               1244.065625   753.0    398101
                Unknown                     1590.000000  1590.0      1590
Transfer        Intact Female                409.398152   123.0   4652810
                Intact Male                  352.124940    76.0   3638507
                Neutered Male               1124.225640   738.0   8480034
                Spayed Female               1095.877083   735.0   6971970
                Unknown                      169.539593    19.0    749365

But it has the unsavory side effect of creating a two-level index. This can be a good time to use `.pivot_table()`.

(There is also a `.pivot()`. For the somewhat subtle differences, see [here](https://stackoverflow.com/questions/30960338/pandas-difference-between-pivot-and-pivot-table-why-is-only-pivot-table-workin).)

In [36]:
# Check it out!
outcomes.pivot_table(index='Outcome Type', columns='Sex upon Outcome', aggfunc=['mean'])

mean                             \
                 Calculated Age in Days                              
Sex upon Outcome          Intact Female  Intact Male Neutered Male   
Outcome Type                                                         
Adoption                     422.344047   476.037194    651.127082   
Died                         350.784195   303.058537   1842.490385   
Disposal                     407.277778   735.935484   1987.777778   
Euthanasia                  1129.880583   904.908472   2228.568925   
Missing                      266.800000   340.846154   1188.285714   
Relocate                     732.000000          NaN   1105.000000   
Return to Owner             1063.130712  1106.971545   1635.703431   
Rto-Adopt                   1455.724138  1503.175000   1255.863071   
Transfer                     409.398152   352.124940   1124.225640   

                                             
                                             
Sex upon Outcome Spayed Female      Unknown  
Outcome Type                                 
Adoption            647.370062   389.964286  
Died               2100.895833   311.353760  
Disposal           2470.166667   449.377907  
Euthanasia         2312.266417   504.882808  
Missing            1262.733333   169.250000  
Relocate            495.000000   612.200000  
Return to Owner    1751.054955   769.637838  
Rto-Adopt          1244.065625  1590.000000  
Transfer           1095.877083   169.539593

# Methods for Combining DataFrames: `.join()`, `.merge()`, `.concat()`

Many ways to combine dataframes! Luckily, pandas has great docs: https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## `.join()`

In [37]:
toy1 = pd.DataFrame([[63, 142], [33, 47]], columns=['age', 'HP'])
toy2 = pd.DataFrame([[63, 100], [33, 200]], columns=['age', 'MP'])

toy1

age   HP
0   63  142
1   33   47

In [38]:
toy2

age   MP
0   63  100
1   33  200

In [39]:
toy1.join(toy2, lsuffix='1', rsuffix='2')

age1   HP  age2   MP
0    63  142    63  100
1    33   47    33  200

If we don't want to keep both, we could set the overlapping column as the index in each DataFrame:

In [40]:
toy1.set_index('age').join(toy2.set_index('age'))

HP   MP
age          
63   142  100
33    47  200

In [41]:
toy1.drop('age', axis=1).join(toy2)

HP  age   MP
0  142   63  100
1   47   33  200

## `.merge()`

Or we could use `.merge()`:

In [42]:
toy1.merge(toy2)

age   HP   MP
0   63  142  100
1   33   47  200

In [43]:
ds_chars = pd.read_csv('data/ds_chars.csv', index_col=0)
ds_chars

name   HP home_state
0    greg  200         WA
1   miles  200         WA
2    alan  170         TX
3  alison  300         DC
4  rachel  200         TX

In [44]:
states = pd.read_csv('data/states.csv', index_col=0)
states

state   nickname     capital
0    WA  evergreen     Olympia
1    TX      alamo      Austin
2    DC   district  Washington
3    OH    buckeye    Columbus
4    OR     beaver       Salem

## The `how` Parameter

This parameter in both `.join()` and `.merge()` tells the compiler what sort of join to effect. We'll cover this in detail when we discuss SQL.

![image showcasing how the how parameter in a join/merge would combine the two datasets, using venn-style diagrams](https://www.datasciencemadesimple.com/wp-content/uploads/2017/09/join-or-merge-in-python-pandas-1.png)
[[Image Source]](https://www.datasciencemadesimple.com/join-merge-data-frames-pandas-python/)

In [47]:
ds_chars.merge(states,
               left_on='home_state',
               right_on='state',
               how='inner')

name   HP home_state state   nickname     capital
0    greg  200         WA    WA  evergreen     Olympia
1   miles  200         WA    WA  evergreen     Olympia
2    alan  170         TX    TX      alamo      Austin
3  rachel  200         TX    TX      alamo      Austin
4  alison  300         DC    DC   district  Washington

In [48]:
ds_chars.merge(states,
               left_on='home_state',
               right_on='state',
               how='outer')

name     HP home_state state   nickname     capital
0    greg  200.0         WA    WA  evergreen     Olympia
1   miles  200.0         WA    WA  evergreen     Olympia
2    alan  170.0         TX    TX      alamo      Austin
3  rachel  200.0         TX    TX      alamo      Austin
4  alison  300.0         DC    DC   district  Washington
5     NaN    NaN        NaN    OH    buckeye    Columbus
6     NaN    NaN        NaN    OR     beaver       Salem

## `pd.concat()`

This method takes a *list* of pandas objects as arguments.

In [49]:
prefs = pd.read_csv('data/preferences.csv', index_col=0)
prefs

cuisine        genre
0     Greek       horror
1    Indian        scifi
2  American      fantasy
3      Thai         tech
4    Indian  documentary

In [50]:
ds_full = pd.concat([ds_chars, prefs])
ds_full

name     HP home_state   cuisine        genre
0    greg  200.0         WA       NaN          NaN
1   miles  200.0         WA       NaN          NaN
2    alan  170.0         TX       NaN          NaN
3  alison  300.0         DC       NaN          NaN
4  rachel  200.0         TX       NaN          NaN
0     NaN    NaN        NaN     Greek       horror
1     NaN    NaN        NaN    Indian        scifi
2     NaN    NaN        NaN  American      fantasy
3     NaN    NaN        NaN      Thai         tech
4     NaN    NaN        NaN    Indian  documentary

`pd.concat()`–– and many other pandas operations –– make use of an `axis` parameter. For this particular method I need to specify whether I want to concatenate the DataFrames *row-wise* (`axis=0`) or *column-wise* (`axis=1`). The default is `axis=0`, so let's override that!

In [51]:
ds_full = pd.concat([ds_chars, prefs], axis=1)
ds_full

name   HP home_state   cuisine        genre
0    greg  200         WA     Greek       horror
1   miles  200         WA    Indian        scifi
2    alan  170         TX  American      fantasy
3  alison  300         DC      Thai         tech
4  rachel  200         TX    Indian  documentary

## Back to the Center

We have Intakes data and we have Outcomes data... time to merge!

In [52]:
# Peek at the outcomes data we already had in here
outcomes.head()

Animal ID   Name            DateTime MonthYear Date of Birth Outcome Type  \
0   A794011  Chunk 2019-05-08 18:20:00  May 2019    2017-05-02    Rto-Adopt   
1   A776359  Gizmo 2018-07-18 16:02:00  Jul 2018    2017-07-12     Adoption   
2   A821648    NaN 2020-08-16 11:38:00  Aug 2020    2019-08-16   Euthanasia   
3   A720371  Moose 2016-02-13 17:59:00  Feb 2016    2015-10-08     Adoption   
4   A674754    NaN 2014-03-18 11:47:00  Mar 2014    2014-03-12     Transfer   

  Outcome Subtype Animal Type Sex upon Outcome Age upon Outcome  \
0             NaN         Cat    Neutered Male          2 years   
1             NaN         Dog    Neutered Male           1 year   
2             NaN       Other          Unknown           1 year   
3             NaN         Dog    Neutered Male         4 months   
4         Partner         Cat      Intact Male           6 days   

                                Breed              Color  \
0              Domestic Shorthair Mix  Brown Tabby/White   
1             Chihuahua Shorthair Mix        White/Brown   
2                             Raccoon               Gray   
3  Anatol Shepherd/Labrador Retriever               Buff   
4              Domestic Shorthair Mix       Orange Tabby   

   Calculated Age in Days  
0                     736  
1                     371  
2                     366  
3                     128  
4                       6

In [53]:
# Read in the intakes data

intakes = pd.read_csv('data/Austin_Animal_Center_Intakes_041122.csv')

# Check out the intakes data
intakes.head()


Animal ID     Name                DateTime     MonthYear  \
0   A786884   *Brock  01/03/2019 04:19:00 PM  January 2019   
1   A706918    Belle  07/05/2015 12:59:00 PM     July 2015   
2   A724273  Runster  04/14/2016 06:43:00 PM    April 2016   
3   A665644      NaN  10/21/2013 07:59:00 AM  October 2013   
4   A682524      Rio  06/29/2014 10:38:00 AM     June 2014   

                        Found Location Intake Type Intake Condition  \
0  2501 Magin Meadow Dr in Austin (TX)       Stray           Normal   
1     9409 Bluegrass Dr in Austin (TX)       Stray           Normal   
2   2818 Palomino Trail in Austin (TX)       Stray           Normal   
3                          Austin (TX)       Stray             Sick   
4        800 Grove Blvd in Austin (TX)       Stray           Normal   

  Animal Type Sex upon Intake Age upon Intake  \
0         Dog   Neutered Male         2 years   
1         Dog   Spayed Female         8 years   
2         Dog     Intact Male       11 months   
3         Cat   Intact Female         4 weeks   
4         Dog   Neutered Male         4 years   

                                   Breed        Color  
0                             Beagle Mix     Tricolor  
1               English Springer Spaniel  White/Liver  
2                            Basenji Mix  Sable/White  
3                 Domestic Shorthair Mix       Calico  
4  Doberman Pinsch/Australian Cattle Dog     Tan/Gray

In [54]:
# Let's try merging on Animal ID
merged = outcomes.merge(intakes, on='Animal ID',suffixes=('_outcome','_income'))

In [55]:
# What was the result?
merged.head()

Animal ID Name_outcome    DateTime_outcome MonthYear_outcome Date of Birth  \
0   A794011        Chunk 2019-05-08 18:20:00          May 2019    2017-05-02   
1   A776359        Gizmo 2018-07-18 16:02:00          Jul 2018    2017-07-12   
2   A821648          NaN 2020-08-16 11:38:00          Aug 2020    2019-08-16   
3   A720371        Moose 2016-02-13 17:59:00          Feb 2016    2015-10-08   
4   A720371        Moose 2016-02-13 17:59:00          Feb 2016    2015-10-08   

  Outcome Type Outcome Subtype Animal Type_outcome Sex upon Outcome  \
0    Rto-Adopt             NaN                 Cat    Neutered Male   
1     Adoption             NaN                 Dog    Neutered Male   
2   Euthanasia             NaN               Other          Unknown   
3     Adoption             NaN                 Dog    Neutered Male   
4     Adoption             NaN                 Dog    Neutered Male   

  Age upon Outcome  ...         DateTime_income MonthYear_income  \
0          2 years  ...  05/02/2019 04:51:00 PM         May 2019   
1           1 year  ...  07/12/2018 12:46:00 PM        July 2018   
2           1 year  ...  08/16/2020 10:10:00 AM      August 2020   
3         4 months  ...  02/08/2016 11:05:00 AM    February 2016   
4         4 months  ...  02/15/2016 10:37:00 AM    February 2016   

                               Found Location      Intake Type  \
0                                 Austin (TX)  Owner Surrender   
1           7201 Levander Loop in Austin (TX)            Stray   
2  Armadillo Rd And Clubway Ln in Austin (TX)         Wildlife   
3       Dove Dr And E Stassney in Austin (TX)            Stray   
4                                 Austin (TX)  Owner Surrender   

  Intake Condition Animal Type_income Sex upon Intake Age upon Intake  \
0           Normal                Cat   Neutered Male         2 years   
1           Normal                Dog     Intact Male          1 year   
2             Sick              Other         Unknown          1 year   
3           Normal                Dog     Intact Male        4 months   
4           Normal                Dog   Neutered Male        4 months   

                         Breed_income       Color_income  
0              Domestic Shorthair Mix  Brown Tabby/White  
1             Chihuahua Shorthair Mix        White/Brown  
2                             Raccoon               Gray  
3  Anatol Shepherd/Labrador Retriever               Buff  
4  Anatol Shepherd/Labrador Retriever               Buff  

[5 rows x 24 columns]

In [56]:
# Check the shape...
merged.shape

(178210, 24)

In [57]:
# And the shapes of the two original dataframes?


Let's discuss/explore: did that work the way we expected?

- 


<details>
    <summary>Observation Notes</summary>

- We went from about 136k rows in each of the dataframes to 176k! Even using an inner join! Something seems off. 
    
    
</details>

In [58]:
# We might want to try something different
# Can we clean something to make a better merge?
clean_outcomes = outcomes.drop_duplicates(subset =['Animal ID'],keep='last')
clean_intakes = intakes.drop_duplicates(subset=['Animal ID'],keep='last')

In [59]:
# Try again
clean_intakes.head()

Animal ID     Name                DateTime     MonthYear  \
0   A786884   *Brock  01/03/2019 04:19:00 PM  January 2019   
1   A706918    Belle  07/05/2015 12:59:00 PM     July 2015   
2   A724273  Runster  04/14/2016 06:43:00 PM    April 2016   
3   A665644      NaN  10/21/2013 07:59:00 AM  October 2013   
4   A682524      Rio  06/29/2014 10:38:00 AM     June 2014   

                        Found Location Intake Type Intake Condition  \
0  2501 Magin Meadow Dr in Austin (TX)       Stray           Normal   
1     9409 Bluegrass Dr in Austin (TX)       Stray           Normal   
2   2818 Palomino Trail in Austin (TX)       Stray           Normal   
3                          Austin (TX)       Stray             Sick   
4        800 Grove Blvd in Austin (TX)       Stray           Normal   

  Animal Type Sex upon Intake Age upon Intake  \
0         Dog   Neutered Male         2 years   
1         Dog   Spayed Female         8 years   
2         Dog     Intact Male       11 months   
3         Cat   Intact Female         4 weeks   
4         Dog   Neutered Male         4 years   

                                   Breed        Color  
0                             Beagle Mix     Tricolor  
1               English Springer Spaniel  White/Liver  
2                            Basenji Mix  Sable/White  
3                 Domestic Shorthair Mix       Calico  
4  Doberman Pinsch/Australian Cattle Dog     Tan/Gray

In [ ]:
clean_outcomes.merge(clean_intakes, on='Animal Type')

# Level Up: Quick Column Name Clean Up Code

Throwing a quick use of a lambda function your way:

In [ ]:
outcomes_renamed = outcomes.rename(columns = lambda x: x.replace(" ", "_").lower())
outcomes_renamed.head()

# Level Up: `pandas.set_option()`

We can adjust how `pandas` works by setting options in advance.

For complete documentation, see [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html).

## Block Scientific Notation

For example, suppose we want to prevent numbers from being displayed in scientific notation.

In [ ]:
df = pd.DataFrame([[1e9, 2e9], [3e9, 4e9]])
df

Then we can use:

In [ ]:
pd.set_option('display.float_format', '{:.2f}'.format)

df

## See More Rows

Or suppose we want `pandas` to show more rows.

In [ ]:
df2 = pd.DataFrame(np.array(range(100)))
df2

In that case we can use:

In [ ]:
pd.set_option('display.max_rows', 100)

df2